### 계약전력합계 : 해당 건물이 계약한 최대 전력량
### 전력수요합계 : 주어진 시간 동안 소비된 총 전력량
### 전력부하량 평균 : 주어진 시간 동안의 평균 전력 부하량

In [2]:
import pandas as pd

In [3]:
train = pd.read_csv(r"C:/Users/DC/OneDrive - 계명대학교/DC/2024/2024_weatherbig/데셋/파생변수_train.csv")

In [4]:
train.columns

Index(['Unnamed: 0', '격자넘버', '날짜', '시간', '공동주택수', '지점번호', '계약전력합계', '전력수요합계',
       '전력부하량평균', '기온', '상대습도', '풍속', '강수량', '체감온도', '요일', '주중주말', '전력기상지수',
       '계절', '년', '월', '일', 'CDH', 'HDH', '불쾌지수', '기온_계절별평균', '상대습도_계절별평균',
       '체감온도_계절별평균', '풍속_계절별평균', '강수량_계절별평균'],
      dtype='object')

In [5]:
train.head()

,Unnamed: 0,격자넘버,날짜,시간,공동주택수,지점번호,계약전력합계,전력수요합계,전력부하량평균,기온,...,월,일,CDH,HDH,불쾌지수,기온_계절별평균,상대습도_계절별평균,체감온도_계절별평균,풍속_계절별평균,강수량_계절별평균
0,0,16914,2020-01-01 01:00:00,1,10,127,15550,2421.96,231.707030,-0.2,...,1,1,0.0,18.2,40.976712,5.715689,61.006934,2.178274,1.963734,0.034498
1,1,14240,2020-01-01 01:00:00,1,12,279,19500,3067.80,258.394386,-1.2,...,1,1,0.0,19.2,37.320704,5.715689,61.006934,2.178274,1.963734,0.034498
2,2,14235,2020-01-01 01:00:00,1,18,822,19674,3204.88,173.613430,-2.3,...,1,1,0.0,20.3,36.546348,5.715689,61.006934,2.178274,1.963734,0.034498
3,3,17495,2020-01-01 01:00:00,1,55,550,135200,19203.08,345.036894,-3.9,...,1,1,0.0,21.9,37.347641,5.715689,61.006934,2.178274,1.963734,0.034498
4,4,11125,2020-01-01 01:00:00,1,61,937,149300,17194.72,300.747194,2.9,...,1,1,0.0,15.1,41.071573,5.715689,61.006934,2.178274,1.963734,0.034498


In [6]:
train.columns

Index(['Unnamed: 0', '격자넘버', '날짜', '시간', '공동주택수', '지점번호', '계약전력합계', '전력수요합계',
       '전력부하량평균', '기온', '상대습도', '풍속', '강수량', '체감온도', '요일', '주중주말', '전력기상지수',
       '계절', '년', '월', '일', 'CDH', 'HDH', '불쾌지수', '기온_계절별평균', '상대습도_계절별평균',
       '체감온도_계절별평균', '풍속_계절별평균', '강수량_계절별평균'],
      dtype='object')

### 계절별 모델 앙상블

#### 응애1

In [7]:
# 필요한 열만 뽑기
train_drop = train.drop(columns = ['Unnamed: 0','날짜','공동주택수','계약전력합계','전력부하량평균',
                                   '년','체감온도_계절별평균','상대습도_계절별평균','전력수요합계'])

# test_drop = test.drop(columns = ['Unnamed: 0','상대습도_계절별평균','체감온도_계절별평균','날짜','년','elect'])

In [8]:
print(len(train_drop.columns))
# print(len(test_drop.columns))

20


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 모델 저장을 위한 딕셔너리 생성
models = {}

# 계절 변수를 원핫인코딩
train_drop = pd.get_dummies(train_drop, columns=['계절'])

# 원핫 인코딩 후 생성된 열 이름 확인
print(train_drop.columns)

Index(['격자넘버', '시간', '지점번호', '기온', '상대습도', '풍속', '강수량', '체감온도', '요일', '주중주말',
       '전력기상지수', '월', '일', 'CDH', 'HDH', '불쾌지수', '기온_계절별평균', '풍속_계절별평균',
       '강수량_계절별평균', '계절_가을', '계절_겨울', '계절_봄', '계절_여름'],
      dtype='object')


In [10]:
spring_tr = train_drop[train_drop['계절_봄']==True]
summer_tr = train_drop[train_drop['계절_여름']==True]
autum_tr = train_drop[train_drop['계절_가을']==True]
winter_tr = train_drop[train_drop['계절_겨울']==True]

In [11]:
test = pd.read_csv(r'C:/Users/DC/OneDrive - 계명대학교/DC/2024/2024_weatherbig/데셋/real2_ele_te.csv')

In [12]:
test.rename(columns={'electric_test.num': '격자넘버'}, inplace=True)
test.rename(columns={'electric_test.tm': '날짜'}, inplace=True)
test.rename(columns={'electric_test.hh24': '시간'}, inplace=True)
test.rename(columns={'electric_test.stn': '지점번호'}, inplace=True)
test.rename(columns={'electric_test.nph_ta': '기온'}, inplace=True)
test.rename(columns={'electric_test.nph_hm': '상대습도'}, inplace=True)
test.rename(columns={'electric_test.nph_ws_10m': '풍속'}, inplace=True)
test.rename(columns={'electric_test.nph_rn_60m': '강수량'}, inplace=True)
test.rename(columns={'electric_test.nph_ta_chi': '체감온도'}, inplace=True)
test.rename(columns={'electric_test.weekday': '요일'}, inplace=True)
test.rename(columns={'electric_test.week_name': '주중주말'}, inplace=True)

In [13]:
test.drop('Unnamed: 0', axis=1, inplace=True)

In [14]:
base_temp = 18.0

# CDH와 HDH 계산 함수
def calculate_cdh(temp, base_temp):
    return max(0, temp - base_temp)

def calculate_hdh(temp, base_temp):
    return max(0, base_temp - temp)

# test 셋
test['CDH'] = test.apply(lambda x: calculate_cdh(x['기온'], base_temp), axis=1)
test['HDH'] = test.apply(lambda x: calculate_hdh(x['기온'], base_temp), axis=1)

# 불쾌지수 컬럼 추가
def calculate_discomfort_index(temperature, humidity):
    return 1.8 * temperature - 0.55 * (1 - humidity / 100) * (1.8 * temperature - 26) + 32

# test 셋
test['불쾌지수'] = test.apply(lambda x: calculate_discomfort_index(x['기온'], x['상대습도']), axis=1)


In [15]:
test

,격자넘버,날짜,시간,지점번호,기온,상대습도,풍속,강수량,체감온도,요일,주중주말,CDH,HDH,불쾌지수
0,4816,2023-01-01 01:00:00,1,752,3.0,68.6,2.9,0.0,-0.1,6,1,0.0,15.0,40.957620
1,4816,2023-01-01 02:00:00,2,752,3.1,69.4,2.7,0.0,0.3,6,1,0.0,14.9,41.016686
2,4816,2023-01-01 03:00:00,3,752,3.6,68.3,2.3,0.0,1.2,6,1,0.0,14.4,41.883312
3,4816,2023-01-01 04:00:00,4,752,4.0,69.2,3.1,0.0,1.1,6,1,0.0,14.0,42.384720
4,4816,2023-01-01 05:00:00,5,752,4.2,69.5,2.5,0.0,2.0,6,1,0.0,13.8,42.653310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829473,12322,2023-12-31 20:00:00,20,901,5.6,70.8,3.2,0.0,5.8,6,1,0.0,12.4,44.636752
2829474,12322,2023-12-31 21:00:00,21,901,5.3,69.1,3.6,0.0,4.5,6,1,0.0,12.7,44.337377
2829475,12322,2023-12-31 22:00:00,22,901,5.1,70.6,3.0,0.0,5.1,6,1,0.0,12.9,43.899794
2829476,12322,2023-12-31 23:00:00,23,901,5.2,69.7,3.8,0.0,5.1,6,1,0.0,12.8,44.133056


In [16]:
test = test.sort_values(by='날짜')
test['날짜'] = pd.to_datetime(test['날짜'])

In [17]:
def assign_season(date):
    if (date.month == 2) or (date.month == 3) or (date.month == 4):
        return '봄'
    elif (date.month == 5) or (date.month == 6) or (date.month == 7):
        return '여름'
    elif (date.month == 8) or (date.month == 9) or (date.month == 10):
        return '가을'
    else:
        return '겨울'

# '계절' 컬럼 생성
test['계절'] = test['날짜'].apply(assign_season)

In [18]:
seasonal_averages = test.groupby('계절').agg({
    '기온':'mean',
    '상대습도':'mean',
    '체감온도':'mean',
    '풍속':'mean',
    '강수량':'mean'
}).reset_index()

seasonal_averages

,계절,기온,상대습도,체감온도,풍속,강수량
0,가을,22.021828,78.373901,23.030252,1.543439,0.169337
1,겨울,5.729009,63.054185,2.239713,2.061995,0.074200
2,봄,9.466310,63.172411,8.188722,1.979861,0.057899
3,여름,21.210946,81.588478,24.043474,1.718530,0.303438


In [19]:
test = test.merge(seasonal_averages, on='계절', suffixes=('', '_계절별평균'))

In [20]:
def assign_date(df, date_column):
    df['년'] = df[date_column].dt.year
    df['월'] = df[date_column].dt.month
    df['일'] = df[date_column].dt.day
    df['시간'] = df[date_column].dt.hour
    return df

test = assign_date(test, '날짜')

In [21]:
test.drop(['날짜','상대습도_계절별평균','년','체감온도_계절별평균'],axis=1, inplace=True)

In [22]:
test = pd.get_dummies(test, columns=['계절'])

In [23]:
spring_te = test[test['계절_봄']==True]
summer_te = test[test['계절_여름']==True]
autum_te = test[test['계절_가을']==True]
winter_te = test[test['계절_겨울']==True]

#### 모델링

##### 봄

In [66]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from supervised.automl import AutoML
import joblib

# 데이터 나누기 (train/test split)
X = spring_tr.drop('전력기상지수', axis=1)
y = spring_tr['전력기상지수']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# AutoML 객체 생성
automl = AutoML(mode="Compete", ml_task="regression")

# 모델 학습
automl.fit(X_train, y_train)

# 예측 수행
predictions = automl.predict(X_test)

# 평가지표 계산
rmse = mean_squared_error(y_test, predictions, squared=False)  # RMSE
mae = mean_absolute_error(y_test, predictions)  # MAE
r2 = r2_score(y_test, predictions)  # R^2

# 결과 출력
print("예측 결과:", predictions)
print("실제 값:", y_test.values)
print("RMSE:", rmse)
print("MAE:", mae)
print("R^2:", r2)

# 모델 평가
score = automl.score(X_test, y_test)
print("모델 기본 평가 점수 (R^2):", score)

# 모델 저장
joblib.dump(automl, 'spring_best_model.pkl')

# 모델 불러오기
loaded_model = joblib.load('spring_best_model.pkl')

# 종속변수 없는 데이터 셋 예측
predictions_new_data = loaded_model.predict(spring_te)

# 새로운 데이터 셋에 대한 예측 결과 출력
print("새로운 데이터 예측 결과:", predictions_new_data)


Linear algorithm was disabled.
AutoML directory: AutoML_1
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree rmse 7.563558 trained in 2.73 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle
* Step simple_algorithms will try to check up to 3 models
1_DecisionTree rmse 7.556686 trained in 25.9 seconds
2_DecisionTree rmse 6.562509 trained in 31.53 seconds
3_DecisionTree rmse 6.562509 trained in 

c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.


예측 결과: [ 80.98797468  89.92494695  89.45553825 ... 125.82018273 113.45292441
 120.74309907]
실제 값: [ 82.31  88.63  88.69 ... 125.52 114.98 121.61]
RMSE: 1.6435969390314444
MAE: 1.1891792937599188
R^2: 0.9925710475060209
모델 기본 평가 점수 (R^2): 0.9925710475060209
새로운 데이터 예측 결과: [109.99533    112.77627345 104.70761811 ... 111.61825256 108.49222237
 113.11735543]


##### 여름

In [67]:
import pandas as pd
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML
import joblib


# 데이터 나누기 (train/test split)
X = summer_tr.drop('전력기상지수', axis=1)
y = summer_tr['전력기상지수']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# AutoML 객체 생성
automl = AutoML(mode="Compete", ml_task="regression")

# 모델 학습
automl.fit(X_train, y_train)

# 예측 수행
predictions = automl.predict(X_test)

# 평가지표 계산
rmse = mean_squared_error(y_test, predictions, squared=False)  # RMSE
mae = mean_absolute_error(y_test, predictions)  # MAE
r2 = r2_score(y_test, predictions)  # R^2

# 결과 출력
print("예측 결과:", predictions)
print("실제 값:", y_test.values)
print("RMSE:", rmse)
print("MAE:", mae)
print("R^2:", r2)

# 모델 평가
score = automl.score(X_test, y_test)
print("모델 기본 평가 점수 (R^2):", score)

# 모델 저장
joblib.dump(automl, 'summer_best_model.pkl')

# 모델 불러오기
loaded_model = joblib.load('summer_best_model.pkl')

# 종속변수 없는 데이터 셋 예측
predictions_new_data = loaded_model.predict(summer_te)

# 새로운 데이터 셋에 대한 예측 결과 출력
print("새로운 데이터 예측 결과:", predictions_new_data)


Linear algorithm was disabled.
AutoML directory: AutoML_2
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree rmse 15.609706 trained in 2.78 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle
* Step simple_algorithms will try to check up to 3 models
1_DecisionTree rmse 15.602251 trained in 25.91 seconds
2_DecisionTree rmse 13.393485 trained in 31.77 seconds
3_DecisionTree rmse 13.393485 traine

c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.


예측 결과: [ 66.55663287  97.9070792  114.48300727 ...  95.51923364  77.05450224
  85.90299379]
실제 값: [ 66.96  97.32 113.85 ...  99.37  76.17  86.43]
RMSE: 2.431823302690932
MAE: 1.7329279960789619
R^2: 0.9926747693512783
모델 기본 평가 점수 (R^2): 0.9926747693512783
새로운 데이터 예측 결과: [ 97.83997207  99.21119495  98.98136316 ... 128.77790039 155.57753158
 138.3586041 ]


##### 가을

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML
import joblib

# 데이터 나누기 (train/test split)
X = autum_tr.drop('전력기상지수', axis=1)
y = autum_tr['전력기상지수']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# AutoML 객체 생성
automl = AutoML(mode="Compete", ml_task="regression")

# 모델 학습
automl.fit(X_train, y_train)

# 예측 수행
predictions = automl.predict(X_test)

# 평가지표 계산
rmse = mean_squared_error(y_test, predictions, squared=False)  # RMSE
mae = mean_absolute_error(y_test, predictions)  # MAE
r2 = r2_score(y_test, predictions)  # R^2

# 결과 출력
print("예측 결과:", predictions)
print("실제 값:", y_test.values)
print("RMSE:", rmse)
print("MAE:", mae)
print("R^2:", r2)

# 모델 평가
score = automl.score(X_test, y_test)
print("모델 기본 평가 점수 (R^2):", score)

# 모델 저장
joblib.dump(automl, 'autum_best_model.pkl')

# 모델 불러오기
loaded_model = joblib.load('autum_best_model.pkl')

# 종속변수 없는 데이터 셋 예측
predictions_new_data = loaded_model.predict(autum_te)

# 새로운 데이터 셋에 대한 예측 결과 출력
print("새로운 데이터 예측 결과:", predictions_new_data)


Linear algorithm was disabled.
AutoML directory: AutoML_1
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree rmse 15.224235 trained in 2.75 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle
* Step simple_algorithms will try to check up to 3 models
1_DecisionTree rmse 15.218044 trained in 25.9 seconds
2_DecisionTree rmse 13.146438 trained in 31.56 seconds
3_DecisionTree rmse 13.146438 trained

c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.


예측 결과: [ 97.18604791 129.31456578  92.89893184 ... 183.38397782 112.42460507
  97.36834642]
실제 값: [100.66 129.47  90.86 ... 186.93 112.05 102.31]
RMSE: 2.6233876943859538
MAE: 1.8409368976552967
R^2: 0.9924197419872597
모델 기본 평가 점수 (R^2): 0.9924197419872597
새로운 데이터 예측 결과: [150.92599387 148.87954355 145.48057102 ... 111.7851913  110.05183085
 112.42679792]


##### 겨울

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML
import joblib


# 데이터 나누기 (train/test split)
X = winter_tr.drop('전력기상지수', axis=1)
y = winter_tr['전력기상지수']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# AutoML 객체 생성
automl = AutoML(mode="Compete", ml_task="regression")

# 모델 학습
automl.fit(X_train, y_train)


# 예측 수행
predictions = automl.predict(X_test)

# 평가지표 계산
rmse = mean_squared_error(y_test, predictions, squared=False)  # RMSE
mae = mean_absolute_error(y_test, predictions)  # MAE
r2 = r2_score(y_test, predictions)  # R^2

# 결과 출력
print("예측 결과:", predictions)
print("실제 값:", y_test.values)
print("RMSE:", rmse)
print("MAE:", mae)
print("R^2:", r2)

# 모델 평가
score = automl.score(X_test, y_test)
print("모델 기본 평가 점수 (R^2):", score)

# 모델 저장
joblib.dump(automl, 'winter_best_model.pkl')

# 모델 불러오기
loaded_model = joblib.load('winter_best_model.pkl')

# 종속변수 없는 데이터 셋 예측
predictions_new_data = loaded_model.predict(winter_te)

# 새로운 데이터 셋에 대한 예측 결과 출력
print("새로운 데이터 예측 결과:", predictions_new_data)


Linear algorithm was disabled.
AutoML directory: AutoML_2
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree rmse 8.185414 trained in 2.9 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle
* Step simple_algorithms will try to check up to 3 models
1_DecisionTree rmse 8.202813 trained in 26.3 seconds
2_DecisionTree rmse 7.289248 trained in 31.67 seconds
3_DecisionTree rmse 7.289248 trained in 3

c:\Users\DC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.


예측 결과: [142.74276255  91.15300142 102.92674005 ...  92.31527215  83.94799052
  69.41509086]
실제 값: [140.46  92.81 103.05 ...  93.18  85.74  69.63]
RMSE: 1.7036031889683647
MAE: 1.2511722418490399
R^2: 0.9927978240157602
모델 기본 평가 점수 (R^2): 0.9927978240157602
새로운 데이터 예측 결과: [ 94.68949314 102.12257068 104.74749024 ... 112.18084126 112.63822186
 109.40718097]


In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# from tqdm import tqdm
# from supervised.automl import AutoML

# # 모델 저장을 위한 딕셔너리 생성
# models = {}

# # 계절별로 데이터 분할 및 모델 학습
# seasons = ['계절_가을', '계절_겨울', '계절_봄', '계절_여름']
# for season in tqdm(seasons, desc="Season Progress"):
#     season_data = train_drop[train_drop[season] == 1]

#     X = season_data.drop(columns=['전력수요합계', '전력기상지수'])
#     y = season_data['전력기상지수']

#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     automl = AutoML(results_path=f"./AutoML_Results_{season}", total_time_limit=3600, mode="Perform")
#     automl.fit(X_train, y_train)

#     models[season] = automl

#     y_pred = automl.predict(X_test)
#     print(f'계절 {season} - MAE:', mean_absolute_error(y_test, y_pred))
#     print(f'계절 {season} - RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
#     print(f'계절 {season} - r2:', r2_score(y_test, y_pred))

Season Progress:   0%|          | 0/4 [00:00<?, ?it/s]

Linear algorithm was disabled.
AutoML directory: ./AutoML_Results_계절_가을
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'ensemble']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 5 models
cannot set WRITEABLE flag to True of this array
Problem during computing permutation importance. Skipping ...
There was an error during 1_Default_LightGBM training.
Please check ./AutoML_Results_계절_가을\errors.md for details.
There was an error during 2_Default_Xgboost training.
Please check ./AutoML_Results_계절_가을\errors.md for details.
There was an error during 3_Default_CatBoost training.
Please check ./AutoML_Results_계절_가을\e

Season Progress:   0%|          | 0/4 [29:35<?, ?it/s]


KeyboardInterrupt: 